In [2]:
import os
%pwd

'/Users/jeromeamos/Desktop/icare/MLops/end_to_end_project/datascienceproject/research'

In [3]:
os.chdir('../')
%pwd

'/Users/jeromeamos/Desktop/icare/MLops/end_to_end_project/datascienceproject'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:  #same element as in config.yaml
    root_dir: Path
    source_URL:Path
    local_data_file:Path
    unzip_dir: Path



In [10]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories



class ConfigurationManager:
    def __init__(self, 
                 config_filepath=CONFIG_FILE_PATH,
                 param_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath) #return a configbox
        self.param = read_yaml(param_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])        

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir   
        )
        return data_ingestion_config

In [9]:
# component-Data ingestion
import os
import urllib.request as request
from src.datascience import logger
import zipfile

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    # Downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL, 
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download with following info : {headers}")
        else:
            logger.info(f"File already exists of size")
            
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    print(data_ingestion_config)
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-01-07 18:26:32,794:  INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-01-07 18:26:32,795:  INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-07 18:26:32,796:  INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-07 18:26:32,796:  INFO: common: created directory at: artifacts]
[2025-01-07 18:26:32,797:  INFO: common: created directory at: artifacts/data_ingestion]
DataIngestionConfig(root_dir='artifacts/data_ingestion', source_URL='https://github.com/krishnaik06/datasets/raw/refs/heads/main/winequality-data.zip', local_data_file='artifacts/data_ingestion_data.zip', unzip_dir='artifacts/data_ingestion')
[2025-01-07 18:26:33,332:  INFO: 587080273: artifacts/data_ingestion_data.zip download with following info : Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981